## Data Cleaning
Data was provided in a wide-format and needs to be converted to a [tidy long-format](https://vita.had.co.nz/papers/tidy-data.html).
This will make it easier to work with in Tableau and programmatically with Python.

I did find that repivotting the revenue and user/visitation variables in the long dataframe worked best for my needs in Tableau. parks-data-pivot.csv is this file and is what was used in Tableau. A dataframe of total monthly revenue and monthly percent change, was also created and used in Tableau for a basic KPI tracker. 

In [24]:
from pathlib import Path

import pandas as pd
import numpy as np

In [25]:
# Create base paths to the data directories
raw_data_path = Path('../data/raw')
processed_data_path = Path('../data/processed')

# Read in the provided parks data
df = pd.read_csv(raw_data_path / 'parks-data.csv')
df.head()

,Date,Springfield Oaks Golf Course Golf Rounds Played,Springfield Oaks Golf Course Golf Revenue,Glen Oaks Golf Course Golf Rounds Played,Glen Oaks Golf Course Golf Revenue,Red Oaks Waterpark Visitors,Red Oaks Waterpark Revenue,Waterford Oaks Waterpark Visitors,Waterford Oaks Waterpark Revenue,Groveland Oaks Campers,Groveland Oaks Campground Revenue,Addison Oaks Campers,Addison Oaks Campground Revenue,Temperature (F),Precipitation (inches)
0,6/1/2022,37,1184,32,1248,134,402,213,852,9,108,10,80,75,0.42
1,6/2/2022,45,1485,42,1344,287,1148,329,1316,13,169,11,110,90,0.07
2,6/3/2022,22,880,26,962,170,680,178,534,11,143,5,50,82,0.36
3,6/4/2022,23,782,30,1050,105,315,164,820,8,88,10,100,77,0.35
4,6/5/2022,46,1794,43,1376,380,1140,289,1156,13,169,17,204,87,0.14


In [26]:
# Initial data cleaning
def tweak_df(df: pd.DataFrame) -> pd.DataFrame:
    return (df
            .rename(columns={'Date':'date',
                             'Springfield Oaks Golf Course Golf Rounds Played':'springfield_oaks_golf_rounds_played',
                             'Springfield Oaks Golf Course Golf Revenue': 'springfield_oaks_golf_revenue',
                             'Glen Oaks Golf Course Golf Rounds Played': 'glen_oaks_golf_rounds_played',
                             'Glen Oaks Golf Course Golf Revenue': 'glen_oaks_golf_revenue',
                             'Red Oaks Waterpark Visitors': 'red_oaks_waterpark_visitors',
                             'Red Oaks Waterpark Revenue': 'red_oaks_waterpark_revenue',
                             'Waterford Oaks Waterpark Visitors': 'waterford_oaks_waterpark_visitors',
                             'Waterford Oaks Waterpark Revenue': 'waterford_oaks_waterpark_revenue',
                             'Groveland Oaks Campers': 'groveland_oaks_campers',
                             'Groveland Oaks Campground Revenue': 'groveland_oaks_campground_revenue',
                             'Addison Oaks Campers': 'addison_oaks_campers',
                             'Addison Oaks Campground Revenue': 'addison_oaks_campground_revenue',
                             'Temperature (F)': 'temp_f',
                             'Precipitation (inches)': 'precip_in',})
            .assign(date=lambda x: pd.to_datetime(x['date'], format='%m/%d/%Y')
            )
    )
      
df_tweaked = tweak_df(df)
df_tweaked.head()

,date,springfield_oaks_golf_rounds_played,springfield_oaks_golf_revenue,glen_oaks_golf_rounds_played,glen_oaks_golf_revenue,red_oaks_waterpark_visitors,red_oaks_waterpark_revenue,waterford_oaks_waterpark_visitors,waterford_oaks_waterpark_revenue,groveland_oaks_campers,groveland_oaks_campground_revenue,addison_oaks_campers,addison_oaks_campground_revenue,temp_f,precip_in
0,2022-06-01,37,1184,32,1248,134,402,213,852,9,108,10,80,75,0.42
1,2022-06-02,45,1485,42,1344,287,1148,329,1316,13,169,11,110,90,0.07
2,2022-06-03,22,880,26,962,170,680,178,534,11,143,5,50,82,0.36
3,2022-06-04,23,782,30,1050,105,315,164,820,8,88,10,100,77,0.35
4,2022-06-05,46,1794,43,1376,380,1140,289,1156,13,169,17,204,87,0.14


In [27]:
def split_parks_and_weather_data(df: pd.DataFrame) -> pd.DataFrame:
    parks_data = df.drop(columns=['temp_f', 'precip_in'])
    weather = df[['date', 'temp_f', 'precip_in']]
    return parks_data, weather

parks_data, weather = split_parks_and_weather_data(df_tweaked)

parks_data.to_csv(processed_data_path / 'parks-data.csv', index=False)  
weather.to_csv(processed_data_path / 'weather.csv', index=False)
weather.head()

,date,temp_f,precip_in
0,2022-06-01,75,0.42
1,2022-06-02,90,0.07
2,2022-06-03,82,0.36
3,2022-06-04,77,0.35
4,2022-06-05,87,0.14


In [28]:
# Convert to long format
def parks_data_wide_to_long(df: pd.DataFrame) -> pd.DataFrame:
    return (df
            .melt(id_vars='date', var_name='intermediate', value_name='value')
            .assign(park_name=lambda df_: (df_['intermediate']
                                   .str.split('_')
                                   .str[0:2]
                                   .str.join(' ').str.title()
                                   ),
                    facility=lambda df_: df_['intermediate'].apply(lambda v: ('golf' if 'golf' in v
                                                                      else 'waterpark' if 'waterpark'in v
                                                                      else 'campground'
                                                                      )
                                                           ),
                    variable=lambda df_: df_['intermediate'].apply(lambda v: ('rounds played' if 'rounds' in v
                                                                      else 'revenue' if 'revenue' in v
                                                                      else 'visitors' if 'visitors' in v
                                                                      else 'campers')
                                                           )
            )
    .drop(columns=['intermediate'])
    .sort_values(['date', 'facility', 'variable'])
    .reset_index(drop=True)
    .reindex(columns=['date', 'park_name', 'facility', 'variable', 'value'])
    )  

parks_data_long = parks_data_wide_to_long(parks_data)
parks_data_long.to_csv(processed_data_path / 'parks-data-long.csv', index=False)
parks_data_long.head()

,date,park_name,facility,variable,value
0,2022-06-01,Groveland Oaks,campground,campers,9
1,2022-06-01,Addison Oaks,campground,campers,10
2,2022-06-01,Groveland Oaks,campground,revenue,108
3,2022-06-01,Addison Oaks,campground,revenue,80
4,2022-06-01,Springfield Oaks,golf,revenue,1184


In [29]:
parks_data_pivot = (parks_data_long
                    .pivot(index=['date', 'park_name', 'facility'],
                           columns='variable',
                           values='value')
                    .reset_index()
)
                    
parks_data_pivot.to_csv(processed_data_path / 'parks-data-pivot.csv', index=False)
parks_data_pivot.head()

variable,date,park_name,facility,campers,revenue,rounds played,visitors
0,2022-06-01,Addison Oaks,campground,10.0,80.0,NaN,NaN
1,2022-06-01,Glen Oaks,golf,NaN,1248.0,32.0,NaN
2,2022-06-01,Groveland Oaks,campground,9.0,108.0,NaN,NaN
3,2022-06-01,Red Oaks,waterpark,NaN,402.0,NaN,134.0
4,2022-06-01,Springfield Oaks,golf,NaN,1184.0,37.0,NaN


In [30]:
parks_data_kpi = (
    parks_data_long
    .set_index('date')
    .groupby(['park_name', 'facility', 'variable'])
    .resample('ME')
    .sum(numeric_only=True)
    .loc[lambda x: x['value'] > 0]
    .reset_index()
    .set_index('date')
    .assign(pct_change= lambda x: np.round(x.groupby(['park_name', 'facility', 'variable'])['value'].pct_change()*100, 1))
    .rename(columns={'value': 'total'})
    .reset_index()
)

parks_data_kpi.to_csv(processed_data_path / 'parks_data_kpi.csv')
parks_data_kpi.head()

,date,park_name,facility,variable,total,pct_change
0,2022-06-30,Addison Oaks,campground,campers,329,NaN
1,2022-07-31,Addison Oaks,campground,campers,376,14.3
2,2022-08-31,Addison Oaks,campground,campers,308,-18.1
3,2023-06-30,Addison Oaks,campground,campers,349,13.3
4,2023-07-31,Addison Oaks,campground,campers,341,-2.3
